In [4]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen 
import re
from bancor_research.bancor_simulator.v3.spec.network import BancorDapp

# Recall from an earlier chapter that we already defined the whitelisted_tokens as follows.
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
v3 = BancorDapp(whitelisted_tokens=whitelisted_tokens)

v3.create_user('Trader')

v3.set_user_balance(tkn_amt=101, tkn_name='eth', user_name='Alice')
v3.set_user_balance(tkn_amt=101, tkn_name='wbtc', user_name='Bob')
v3.set_user_balance(tkn_amt=10001, tkn_name='link', user_name='Charlie')
v3.set_user_balance(tkn_amt=2000, tkn_name='bnt', user_name='Trader')
v3.set_user_balance(tkn_amt=3, tkn_name='link', user_name='Trader')

# The actual deposits.

v3.deposit(tkn_amt=100, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=10000, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=100, tkn_name='wbtc', user_name='Bob')

v3.deposit(tkn_amt=1, tkn_name='eth', user_name='Alice')

v3.deposit(tkn_amt=1, tkn_name='link', user_name='Charlie')

v3.deposit(tkn_amt=1, tkn_name='wbtc', user_name='Bob')

v3.dao_msig_init_pools(whitelisted_tokens, 'bnt')

v3.trade(tkn_amt=2000, source_token='bnt',  target_token='link', user_name='Trader', timestamp=0)

def read_url(url): 
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

Bootstrap requirements met for eth
Bootstrap requirements met for wbtc
Bootstrap requirements met for link


In [5]:
v3.describe(decimals=6)

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,bnt=20000.000000 eth=20.000000,bnt=2000.000000,bnt=0.000000,bnbnt=0.000000,bnt=4.395604,bnt=0.000000,bnbnt=20000.000000
1,bnt=20000.000000 wbtc=1.250000,eth=101.000000,eth=101.000000,bneth=101.000000,,eth=0.000000,
2,bnt=21995.604396 link=3033.333333,wbtc=101.000000,wbtc=101.000000,bnwbtc=101.000000,,wbtc=0.000000,
3,,link=9701.000000,link=10003.424242,bnlink=10001.000000,,link=0.000000,


# Trading and Fees - TKN to BNT

Assume now that a trader wants to perform the opposite action, perhaps to close an arbitrage opportunity left open by the previous swap. The trader sends 302.9981 LINK into the vault, and the vault sends 1977.6155 BNT to the trader. As before, the trader’s intuition agrees with the changing state of the vault balances; however, the changes in the trading liquidity and staked balances require closer examination.

In [6]:
v3.trade(tkn_amt=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestamp=0)

v3.describe(decimals=4)

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,bnt=20000.0000 eth=20.0000,bnt=22.3847,bnt=15.9807,bnbnt=0.0000,bnt=8.3908,bnt=0.0000,bnbnt=20000.0000
1,bnt=20000.0000 wbtc=1.2500,eth=101.0000,eth=101.0000,bneth=101.0000,,eth=0.0000,
2,bnt=20013.9939 link=3336.3314,wbtc=101.0000,wbtc=101.0000,bnwbtc=101.0000,,wbtc=0.0000,
3,,link=10003.9981,link=10003.4242,bnlink=10001.0000,,link=0.0000,


In this trade, the target asset is BNT. Therefore, there is no need to perform a second, virtual swap at the end of the trade. Instead, the vortex fee is taken directly from the effective fee. Importantly, this results in a slightly reduced change in the growth of the BNT trading liquidity as judged from the trader's perspective. The trader's fee totals 19.9759 BNT, of which 3.9952 BNT becomes the property of the Bancor Vortex. The difference (15.9807 BNT) is added to the BNT staking ledger, and causes the value of the bnBNT pool token to appreciate.

Therefore, during the trade the following components of the system were changed:

1. The vault balances. The change in the vault is always in agreement with the trader's intuition. In this case, the trader sent 302.99814 LINK into the vault, and received 1977.615545 BNT from it. Therefore, the vault balance of LINK necessarily increased by 302.99814 LINK, and decreased by 1977.615545 BNT.

2. The staking ledger. Trading fees are always taken from the target asset. Therefore, the value of the bnBNT pool token must be appreciated by 80% of the fee apparent to the trader (i.e. the total fee - vortex rate). A total of 15.980731 BNT tokens are added to the staked amount, which becomes the property of the bnBNT pool token holders (at this point in the narrative, is the protocol only).

3. The vortex ledger. A portion of the fee paid by the trader is effectively collected, and added to the vortex ledger.

4. The trading liquidity. The apparent disagreement between the intuitive result, and the one obtained, is accounted for by the effect of the BNT collection by the vortex.

The change in the BNT and LINK trading liquidity can be expressed as follows:

![](https://lh3.googleusercontent.com/fdNz0QqFKQ0CaabnT7t8pfphTCylaW4ox8YEPNF_i662Xgt3CECetm3GRt34YJ5orHz2zXPPRNX6uIPOlLX1PwSjCXaWmCCcAYrpaeTS0gip1FGZTjOlcOqOACtUJAm-dtXGtsVH)

![](https://lh6.googleusercontent.com/femAV9RrDUJnonpzPV4mevFeQYL1F5RkUXktaroDl78ixkfTwxssW0mglLr0eS_KvC6l0ZewOMX0oQKmi-cnXRpe69q9wg3xich5UXF46qbXEPP9pRzN1xZ94I_dmXnoPRVX2vS1)

![](https://lh6.googleusercontent.com/Y5Isbbj4UjPyamIcti-mUZNAjYiopquxluK2rM0AXAkpqP6ZYhpTU9Ad447Z334-q5b_a2yXXp5oSCBf7e_a3ZqmpG_O9xU3SPfSOtAK8vwOW9cOWBe4VL0bWDFOFzN9k-sd7Veh)

where a1† and a1 are the BNT trading liquidity balances of the pool after and before the trade, respectively, b1† and b1 are the LINK trading liquidity balances of the pool after and before the trade, respectively, d1 is the pool fee (e.g. 0.01, or 1%) and e is the Vortex rate (e.g. 0.2, or 20%). The number of LINK tokens being sent into the vault is x, and the number of BNT tokens sent back to the trader is bntOut. The change in the TKN trading liquidity (and therefore the update to the available LINK trading liquidity) is unchanged from the standard case. The calculation of the fee awarded to liquidity providers, denominated in BNT and added to the staking ledger is:

![](https://lh4.googleusercontent.com/L37q1CVjSJ4-I_j8q_cdhXbxgP9oZ7PXkJ3UVzH67xmPyj9fHUucHYQVBt6wkDnYidSawiK4BIFDbLf1exJc_p7PEDPBgim4XAHRLtBk9IaGCvblhyAXfxnYAk0EcAsJ_CGx_ai_)

And the calculation for the fee given to the Bancor Vortex, denominated exclusively in BNT and added to the vortex ledger is:

![](https://lh3.googleusercontent.com/0sr_cE39asE2B1XQGhuD3whJwUP_yhjwv3QswhtG-nPmfTLQGFBgScf8HAtEyWnGiapYkaTkc5qhdCW0DlZzHDJTRWaTFtYB3Ynn2uij2Tga0czOs-gHFMii521VkSNlLk7KsPDf)